<a href="https://colab.research.google.com/github/radicalcoder/ML-Project/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import matplotlib
import matplotlib.pyplot as plt
import skimage
import skimage.data
import skimage.transform
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import itertools
from keras import models
from keras.preprocessing import image
from PIL import Image
from keras.models import Sequential
import keras
import PIL
from skimage import io
from skimage import color
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout



# To read the data directory from repository on github.
%matplotlib inline
!git clone -l -s https://github.com/radicalcoder/ML-Project.git data
%cd data
!ls

Using TensorFlow backend.


Cloning into 'data'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 7412 (delta 38), reused 0 (delta 0), pack-reused 7349
Receiving objects: 100% (7412/7412), 248.14 MiB | 37.65 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Checking out files: 100% (7225/7225), done.
/content/data
BelgiumTSC_Testing   MLP_final_initialdraft.ipynb  README.md
BelgiumTSC_Training  MLP.ipynb			   traffic.py


In [0]:
def data_loader(data_dir):
    """
    The data set is read into two lists, as follows:
    list labels[] -> a list of numbers, where each number represents a unique image label.
    list images[] -> a list of arrays, where each array represents a single image.
    
    The data set directory contains multiple subdirectories. We read all these 
    subdirectories into a list subdirs[]. Since the subfolders are read in a random order, the list
    is also in a random order. We then sort this array using the sort() function.
    """
    
    subdirs = []
    for i in os.listdir(data_dir):
      if os.path.isdir(data_dir + i):   # There may be .txt files in the data_dir hence we need to check
        subdirs.append(i)

    subdirs.sort()
    
    """
    Data is now collected into two lists - labels & images - while looping through the 
    list subdirs[].    
    """

    labels = []
    images = []
    for i in subdirs:
        label_dir = data_dir + i
        #print(label_dir)
        f_names=[]
        for f in os.listdir(label_dir):
          if f.endswith(".ppm"):   # only files that end in .ppm are image files
            f_names.append(label_dir + '/' + f)
        # print(f_names)
        """
        Now for each label, append all its images into the list images[] and 
        append its label number in the list labels[]. 
        """
        
        for f in f_names:
            images.append(skimage.data.imread(f))   #use parameter as_grey??
            labels.append(int(i))
    return labels, images


# Loading the training and testing dataset using the function data_loader().
training_data_dir = "BelgiumTSC_Training/Training/"
testing_data_dir = "BelgiumTSC_Testing/Testing/"

training_labels, training_images = data_loader(training_data_dir)
testing_labels, testing_images = data_loader(testing_data_dir)
#print(labels)

In [3]:
print("No of unique Labels in Training Data: ", len(set(training_labels)), "\nTotal number of Images in Training Data: ", len(training_images)) 
print("No of unique Labels in Testing Data: ", len(set(testing_labels)), "\nTotal number of Images in Testing Data: ", len(testing_images)) 

No of unique Labels in Training Data:  62 
Total number of Images in Training Data:  4575
No of unique Labels in Testing Data:  53 
Total number of Images in Testing Data:  2520


In [0]:
def display_images_and_labels(images, labels):
    #Display the first image of each label.
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("Label {0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image,cmap="gray")
    plt.show()


In [9]:
#Training Data
reshape_images_training=[]
grayscale_images_training=[]

for i in training_images:
  reshape_images_training.append(skimage.transform.resize(i,(32,32), mode ='constant'))

reshape_images_training= np.array(reshape_images_training)
#reshape_images_training = reshape_images_training.astype('float32')
print(reshape_images_training.shape)

for i in reshape_images_training:
  grayscale_images_training.append(color.rgb2gray(i))
  
grayscale_images_training= np.array(grayscale_images_training)  #Keras expects our images and labels to be in numpy arrays
#grayscale_images_training = reshape_images_training.astype('float32')

print("gray")
print(grayscale_images_training[0].shape)

image_expanded_training = grayscale_images_training[:, :, :, np.newaxis]  #converting images from 32x32 to 32x32x1
grayscale_images_training=image_expanded_training

print(grayscale_images_training[0].shape)

"""

#display_images_and_labels(training_images,training_labels)
#display_images_and_labels(grayscale_images_training, training_labels)

print("Grayscale Training images")
print(len(grayscale_images_training))
for i in grayscale_images_training[:10]:
  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))

print("Reshaped Training Images ")

for i in reshape_images_training[:10]:
  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))
  
"""

(4575, 32, 32, 3)
gray
(32, 32)
(32, 32, 1)


'\n\n#display_images_and_labels(training_images,training_labels)\n#display_images_and_labels(grayscale_images_training, training_labels)\n\nprint("Grayscale Training images")\nprint(len(grayscale_images_training))\nfor i in grayscale_images_training[:10]:\n  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))\n\nprint("Reshaped Training Images ")\n\nfor i in reshape_images_training[:10]:\n  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))\n  \n'

In [10]:
#Testing Data
reshape_images_testing=[]
grayscale_images_testing=[]

for i in testing_images:
  reshape_images_testing.append(skimage.transform.resize(i,(32,32), mode ='constant'))

reshape_images_testing= np.array(reshape_images_testing)
#reshape_images_testing = reshape_images_testing.astype('float32')
print(reshape_images_testing.shape)

for i in reshape_images_testing:
  grayscale_images_testing.append(color.rgb2gray(i))
  
grayscale_images_testing= np.array(grayscale_images_testing)
#grayscale_images_testing = reshape_images_testing.astype('float32')

print("gray")
print(grayscale_images_testing[0].shape)

image_expanded_testing = grayscale_images_testing[:, :, :, np.newaxis]  #converting imags from 32x32 to 32x32x1
grayscale_images_testing=image_expanded_testing

print(grayscale_images_testing[0].shape)

  

#display_images_and_labels(testing_images,testing_labels)
#display_images_and_labels(grayscale_images_testing, testing_labels)

"""

print("Grayscale testing images")
print(len(grayscale_images_testing))
for i in grayscale_images_testing[:10]:
  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))

print("Reshaped testing Images ")

for i in reshape_images_testing[:10]:
  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))

"""

(2520, 32, 32, 3)
gray
(32, 32)
(32, 32, 1)


'\n\nprint("Grayscale testing images")\nprint(len(grayscale_images_testing))\nfor i in grayscale_images_testing[:10]:\n  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))\n\nprint("Reshaped testing Images ")\n\nfor i in reshape_images_testing[:10]:\n  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))\n\n'

In [13]:
#print(training_labels)
from keras.utils import to_categorical
#one-hot encode target column
labels_train = to_categorical(training_labels)
labels_test = to_categorical(testing_labels)
print(labels_train.shape)
print("\n")
print(type(labels_train))
print(training_labels[500])
print(labels_train[500])

#print(testing_labels)
print(labels_test.shape)
print("\n")
print(type(labels_test))
print(testing_labels[500])
print(labels_test[500])

(4575, 62)


<class 'numpy.ndarray'>
13
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(2520, 62)


<class 'numpy.ndarray'>
18
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model_gen = Sequential()
model_gen.add(Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=grayscale_images_training[0].shape))  #When using this layer as the first layer in a model, we provide the keyword argument "input_shape"
#model_gen.add(Conv2D(32, (3, 3), activation='relu'))
model_gen.add(MaxPooling2D(pool_size=(2, 2)))
model_gen.add(Dropout(0.25))
 
model_gen.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
#model_gen.add(Conv2D(64, (3, 3), activation='relu'))
model_gen.add(MaxPooling2D(pool_size=(2, 2)))
model_gen.add(Dropout(0.25))
 
model_gen.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
#model_gen.add(Conv2D(64, (3, 3), activation='relu'))
model_gen.add(MaxPooling2D(pool_size=(2, 2)))
model_gen.add(Dropout(0.25))
 
model_gen.add(Flatten())
model_gen.add(Dense(512, activation='relu')) # we can drop 
model_gen.add(Dropout(0.5))                  # this layers
model_gen.add(Dense(512, activation='relu'))
model_gen.add(Dropout(0.5))
model_gen.add(Dense(62, activation='softmax'))

In [0]:
model_gen.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

# we can use image augmentation
# basically it needs to redifine for normal actual scores like 0.9 of accuracy and more
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=False,
    rotation_range=12,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)

datagen.fit(grayscale_images_training)


%time
epochs=50
batch_size = 20
history_generator = model_gen.fit_generator(datagen.flow(grayscale_images_training, labels_train, batch_size=batch_size),
                    epochs=epochs, steps_per_epoch=500 , verbose=1,
                    validation_data=(grayscale_images_testing, labels_test))